In [11]:
!pip3 install matplotlib
!pip3 install pygame

/home/siddhant/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/siddhant/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/siddhant/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/siddhant/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [12]:
import random
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt
from itertools import combinations
import pygame

In [13]:
height = 150
width = 150
population_current = 0
helpful_pop = 0
unhelpful_pop = 0
tittat_pop = 0
ghost_val = 8
canteen_no = 100
helpful_initial = 100
tittat_initial = 100
unhelpful_initial = 100
hours_per_day = 24
canteen_food = 10
no_of_days = 10
reproduction_threshold = 12
history_size = 5
population_initial = helpful_initial + tittat_initial + unhelpful_initial
block_size = 5

In [14]:
macpen_list = []
helpful_macpens = []
unhelpful_macpens = []
tittat_macpens = []
macpens_loc_dict = {}
canteens_loc_dict = {}
helpful_macpens_dict = {}
tittat_macpens_dict = {}
unhelpful_macpens_dict = {}

# initialize the macpens_loc_dict with empty lists with keys as the location of the macpens
for i in range(height):
  for j in range(width):
      location_id = i*height + j
      macpens_loc_dict[location_id] = []

for i in range(height):
  for j in range(width):
      location_id = i*height + j
      canteens_loc_dict[location_id] = []

In [15]:
class canteen(pygame.sprite.Sprite):
    x = None
    y = None
    food_per_t_per_m = None
    def __init__(self):
        self.image = pygame.Surface([block_size, block_size])
        self.image.fill((51, 51, 255))
        self.rect = self.image.get_rect()
        
    def draw(self, surface):
        pygame.draw.rect(surface, (51, 51, 255), (self.rect.x, self.rect.y, block_size, block_size))

In [16]:
class history:
    x = None
    y = None
    score = None
    def __init__(self, x, y, score):
        self.x = x
        self.y = y
        self.score = score

In [17]:
class macpen(pygame.sprite.Sprite):
    x = None
    y = None
    Type = None
    food = 0
    history_list = None
    ID = None
    donate = None

    def __init__(self, color):
        super().__init__()
        self.image = pygame.Surface([block_size, block_size])
        self.image.fill(color)
        self.rect = self.image.get_rect()
        self.color = color
    
    def move(self):
        up = True
        down = True
        left = True
        right = True
        if (self.x==0): 
            left = False
        if (self.x==width-1): 
            right = False
        if (self.y==0): 
            down = False
        if (self.y==height-1): 
            up = False
        X = []
        Y = []
        count = 0
        if left==True:
            X.append(-1)
            Y.append(0)
            count = count + 1
        if right==True:
            X.append(1)
            Y.append(0)
            count = count + 1
        if up==True:
            X.append(0)
            Y.append(1)
            count = count + 1
        if down==True:
            X.append(0)
            Y.append(-1)
            count = count + 1
        i = random.randrange(0,count)
        self.x = self.x + X[i]
        self.y = self.y + Y[i]
        self.rect.x = self.x*block_size
        self.rect.y = self.y*block_size
        h = history(self.x, self.y, 0)
        self.history_list.append(h)
        
    def interact(self,m):
        excess1 = self.food - ghost_val
        excess2 = m.getExcess()
        
        if(excess1*excess2 > 0):
            pass
        elif(excess1*excess2 < 0):
            if(excess1 > 0 and self.Type == "helpful"):
                require = abs(excess2)
                self.food -= min(excess1, require)
                m.food += min(excess1, require)
                m.history_list[-1].score = m.history_list[-1].score + min(excess1, require)
                self.history_list[-1].score = self.history_list[-1].score - min(excess1, require)
            elif(excess2 > 0 and m.Type == "helpful"):
                require = abs(excess1)
                self.food += min(excess2, require)
                m.food -= min(excess2, require)
                m.history_list[-1].score = m.history_list[-1].score - min(excess2, require)
                self.history_list[-1].score = self.history_list[-1].score + min(excess2, require)
            elif(excess1 > 0 and m.Type == "tittat" and m.donate >= 0):
                require = abs(excess2)
                self.food -= min(excess1, require)
                m.food += min(excess1, require)
                m.history_list[-1].score = m.history_list[-1].score + min(excess1, require)
                self.history_list[-1].score = self.history_list[-1].score - min(excess1, require)
                if(m.Type == "helpful" or m.Type == "unhelpful"):
                    self.donate += m.donate
                elif(m.Type == "tittat" and m.donate > 0):
                    self.donate += 1
                elif(m.Type == "tittat" and m.donate < 0):
                    self.donate -= 1
            elif(excess2 > 0 and m.Type == "tittat" and self.donate >= 0):
                require = abs(excess1)
                m.food -= min(excess2, require)
                self.food += min(excess2, require)
                m.history_list[-1].score = m.history_list[-1].score - min(excess2, require)
                self.history_list[-1].score = self.history_list[-1].score + min(excess2, require)
                if(self.Type == "helpful" or self.Type == "unhelpful"):
                    m.donate += self.donate
                elif(self.Type == "tittat" and self.donate > 0):
                    m.donate += 1
                elif(self.Type == "tittat" and self.donate < 0):
                    m.donate -= 1
        else:
            pass
    
    def can_reproduce(self):
        if self.food>=reproduction_threshold:
            return True
        else:
            return False
    
    def setPosition(self, x, y):
        self.x = x
        self.y = y
    
    def getExcess(self):
        return self.food - ghost_val
    
    def getPosition(self):
        return (self.x, self.y)
    
    def getLocId(self):
        return (self.y)*height + self.x
    
    def draw(self, surface):
        pygame.draw.rect(surface, self.color, (self.rect.x, self.rect.y, block_size, block_size))

    def update(self):
        self.draw(screen)

In [18]:
ID = 0

In [19]:
class macpen_helpful(macpen):
    
    def __init__(self):
        super().__init__((0, 255, 0))
        self.history_list = []
        self.Type = "helpful"
        self.ID = ID
        self.donate = 1
        
    def reproduce(self):
        m = macpen_helpful()
        
        self.food = self.food/2;
        m.food = self.food
        
        m.x = self.x
        m.y = self.y
        m.Type = "helpful"
        h = history(self.x, self.y, 0)
        m.history_list.append(h)
        return m      
       
class macpen_tittat(macpen): 
    
    def __init__(self):
        super().__init__((255, 255, 0))
        self.history_list = []
        self.Type = "tittat"
        self.ID = ID
        self.donate = 0
    
    def reproduce(self):
        m = macpen_tittat()
        
        self.food = self.food/2;
        m.food = self.food
        
        m.x = self.x
        m.y = self.y
        m.Type = "tittat"
        h = history(self.x, self.y, 0)
        m.history_list.append(h)
        
        return m 

class macpen_unhelpful(macpen):
    
    def __init__(self):
        super().__init__((255, 0, 0))
        self.history_list = []
        self.Type = "unhelpful"
        self.ID = ID
        self.donate = -1
    
    def reproduce(self):
        m = macpen_unhelpful()
        
        self.food = self.food/2;
        m.food = self.food
        
        m.x = self.x
        m.y = self.y
        m.Type = "unhelpful"
        h = history(self.x, self.y, 0)
        m.history_list.append(h)
        
        return m 

In [20]:
canteen_list = []
X = random.sample(range(0,height*width),canteen_no)
print(X)
# for i in range(canteen_no):
#     c = canteen()
#     c.x = random.randint(1,width)
#     c.y = random.randint(1, height)
#     c.food_per_t_per_m = canteen_food       
#     loc_id = c.y*height + c.x
#     canteens_loc_dict[loc_id].append(c)

for i in X:
    c = canteen()
    c.x = i%height
    c.y = i//height
    c.food_per_t_per_m = canteen_food
    canteens_loc_dict[i].append(c)

print(len(canteens_loc_dict))

[16398, 6391, 11367, 10101, 6519, 7634, 12080, 9939, 2672, 20856, 9233, 4045, 22277, 19841, 2045, 7876, 11587, 17081, 4713, 1157, 21292, 7717, 2354, 5936, 4304, 9185, 21780, 19165, 9423, 17696, 18316, 2341, 7414, 8501, 12439, 1649, 5093, 22042, 4405, 20150, 20160, 19288, 10877, 4660, 13668, 10244, 6419, 10573, 8065, 16258, 2704, 5016, 6702, 19784, 20369, 17413, 7573, 4452, 6701, 2853, 15493, 318, 19070, 8167, 8081, 7985, 17027, 6471, 18377, 17206, 8825, 13397, 10686, 12363, 11234, 1848, 16017, 1133, 14696, 15943, 14509, 21206, 4165, 1842, 2150, 17388, 14397, 12455, 5807, 16837, 7713, 12369, 105, 13317, 14081, 3100, 6352, 17491, 10136, 8330]
22500


In [21]:
for i in range(helpful_initial):
    mh = macpen_helpful()
    x = random.randint(0,width-1)
    y = random.randint(0,height-1)
    mh.setPosition(x, y)
    mh.ID = ID
    h = history(x,y,0)
    mh.history_list.append(h)
    # macpen_list.append(mh)
    loc_id = y*height + x
    macpens_loc_dict[loc_id].append(mh)
    # helpful_macpens.append(mh)
    helpful_macpens_dict[mh] = loc_id
    population_current = population_current + 1
    helpful_pop = helpful_pop + 1
    ID = ID + 1
    
for i in range(tittat_initial):
    mt = macpen_tittat()
    x = random.randint(0,width-1)
    y = random.randrange(0,height-1)
    mt.setPosition(x, y)
    mt.ID = ID
    h = history(x,y,0)
    mt.history_list.append(h)
    # macpen_list.append(mt)
    loc_id = y*height + x
    macpens_loc_dict[loc_id].append(mt)
    # tittat_macpens.append(mt)
    tittat_macpens_dict[mt] = loc_id
    population_current = population_current + 1
    tittat_pop = tittat_pop + 1
    ID = ID + 1
    
for i in range(unhelpful_initial):
    mu = macpen_unhelpful()
    x = random.randint(0,width-1)
    y = random.randint(0,height-1)
    mu.setPosition(x, y)
    mu.ID = ID
    h = history(x,y,0)
    mu.history_list.append(h)
    # macpen_list.append(mu)
    loc_id = y*height + x
    macpens_loc_dict[loc_id].append(mu)
    # unhelpful_macpens.append(mu)
    unhelpful_macpens_dict[mu] = loc_id
    population_current = population_current + 1
    unhelpful_pop = unhelpful_pop + 1
    ID = ID + 1


print("Initial Population: ", population_initial)
print("Helpful Population: ", helpful_initial)
print("Tittat Population: ", tittat_initial)
print("Unhelpful Population: ", unhelpful_initial)

Initial Population:  300
Helpful Population:  100
Tittat Population:  100
Unhelpful Population:  100


In [22]:
count = 0
class Interactions():
    total = 0
    h_h = 0
    h_t = 0
    h_u = 0
    t_t = 0
    t_u = 0
    
    def __init__(self):
        self.total = 0
        self.h_t = 0
        self.h_u = 0
        self.t_u = 0
    
    def Print(self):
        print("Total Interactions: ", self.total)
        print("Helpful - Tittat Interactions: ", self.h_t)
        print("Helpful - Unhelpful Interactions: ", self.h_u)
        print("Tittat - Unhelpful Interactions: ", self.t_u)
        print("Helpful - Tittat Interactions: ", self.h_t)
        print("Helpful - Unhelpful Interactions: ", self.h_u)

In [23]:
pygame.init()
screen = pygame.display.set_mode((width*block_size, height*block_size))
font = pygame.font.Font(None, 30)

for iterations in range(76):
    print("================== Day: ", iterations//hours_per_day, "Hour: ", iterations%hours_per_day, "================");
    if 1:
        time_surface = font.render(f"Day: {iterations // hours_per_day}, Hour: {iterations % hours_per_day}", True, (255, 255, 255))
        total_pop_surface = font.render(f"Total Population: {len(helpful_macpens_dict) + len(tittat_macpens_dict) + len(unhelpful_macpens_dict)}", True, (255, 255, 255))
        helpful_pop_surface = font.render(f"Helpful: {len(helpful_macpens_dict)}", True, (255, 255, 255))
        tittat_pop_surface = font.render(f"Tit for Tat: {len(tittat_macpens_dict)}", True, (255, 255, 255))
        unhelpful_pop_surface = font.render(f"Unhelpful: {len(unhelpful_macpens_dict)}", True, (255, 255, 255))
        # time.sleep(1)
        # print
        # clear_output(wait=True)
        # X = []
        # Y = []
        # for m in macpen_list:
        #     X.append(m.x)
        #     Y.append(m.y)
        # plt.scatter(X,Y,color='red',s=5)
        # X = []
        # Y = []
        # for c in canteen_list:
        #   X.append(c.x)
        #   Y.append(c.y)
        # plt.scatter(X,Y,color='blue',marker='s', s=100)
        # plt.xlim(1,width)
        # plt.ylim(1,height)
        # plt.show()
        print("Population: ", len(helpful_macpens_dict) + len(tittat_macpens_dict) + len(unhelpful_macpens_dict))
        print("Helpful Population: ", len(helpful_macpens_dict))
        print("Tittat Population: ", len(tittat_macpens_dict))
        print("Unhelpful Population: ", len(unhelpful_macpens_dict))
        if(len(helpful_macpens_dict) + len(tittat_macpens_dict) + len(unhelpful_macpens_dict) > 80000):
            print("population reached above 80000")
            break
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
    # if(len(macpen_list) > 4000):
    #     print("population reached above 4000")
    #     break
    # Feed
    # for c in canteen_list:
    #     head = 5
    #     for m in macpen_list:
    #         if m.x==c.x and m.y == c.y:
    #             m.food = m.food + canteen_food
    #             m.history_list[len(m.history_list)-1].score = m.history_list[len(m.history_list)-1].score + canteen_food

    # Feed
    for m in helpful_macpens_dict:
        if(len(canteens_loc_dict[m.getLocId()]) != 0):
            m.food = m.food + canteen_food
            m.history_list[len(m.history_list)-1].score = m.history_list[len(m.history_list)-1].score + canteen_food
    for m in tittat_macpens_dict:
        if(len(canteens_loc_dict[m.getLocId()]) != 0):
            m.food = m.food + canteen_food
            m.history_list[len(m.history_list)-1].score = m.history_list[len(m.history_list)-1].score + canteen_food
    for m in unhelpful_macpens_dict:
        if(len(canteens_loc_dict[m.getLocId()]) != 0):
            m.food = m.food + canteen_food
            m.history_list[len(m.history_list)-1].score = m.history_list[len(m.history_list)-1].score + canteen_food

    # Interact ##### deprecated #####
    # for (i, m1) in enumerate(macpen_list):
    #     for (j, m2) in enumerate(macpen_list[i+1:]):
    #         if(m1.getPosition() == m2.getPosition()):
    #             # interactions = interactions + 1
    #             # print("interacting", m1.ID, m2.ID)
    #             m1.interact(m2)
    
    # new interaction method
    for loc in macpens_loc_dict:
        if(macpens_loc_dict[loc] != []):
            for(m1, m2) in combinations(macpens_loc_dict[loc], 2):
                m1.interact(m2)
    # Move
    # for m in macpen_list:
    #     m.move()
    
    # Move in dict
    for m in macpens_loc_dict:
        for macpen in macpens_loc_dict[m]:
            # print("x: ", macpen.x, "y: ", macpen.y)
            prev_loc_id = macpen.getLocId()
            # print("prev_loc_id: ", prev_loc_id)
            macpen.move()
            # print("x: ", macpen.x, "y: ", macpen.y)
            new_loc_id = macpen.getLocId()
            macpens_loc_dict[prev_loc_id].remove(macpen)
            # print("prev_loc_id: ", prev_loc_id, "new_loc_id: ", new_loc_id)
            macpens_loc_dict[new_loc_id].append(macpen)
            if(macpen.Type == "helpful"):
                helpful_macpens_dict[macpen] = new_loc_id
            if(macpen.Type == "tittat"):
                tittat_macpens_dict[macpen] = new_loc_id
            if(macpen.Type == "unhelpful"):
                unhelpful_macpens_dict[macpen] = new_loc_id
                
    # Reproduce
    new_ones = []
    for m in helpful_macpens_dict:
        if (m.can_reproduce()):
            m1 = m.reproduce()
            m1.ID = ID
            macpens_loc_dict[m.getLocId()].append(m1)
            # helpful_macpens_dict[m1] = m1.getLocId()
            new_ones.append(m1)
            helpful_pop = helpful_pop + 1
            ID = ID + 1
    for m in tittat_macpens_dict:
        if(m.can_reproduce()):
            m1 = m.reproduce()
            m1.ID = ID
            macpens_loc_dict[m.getLocId()].append(m1)
            # tittat_macpens_dict[m1] = m1.getLocId()
            new_ones.append(m1)
            tittat_pop = tittat_pop + 1
            ID = ID + 1
        
    for m in unhelpful_macpens_dict:
        if(m.can_reproduce()):
            m1 = m.reproduce()
            m1.ID = ID
            macpens_loc_dict[m.getLocId()].append(m1)
            # unhelpful_macpens_dict[m1] = m1.getLocId()
            new_ones.append(m1)
            unhelpful_pop = unhelpful_pop + 1
            ID = ID + 1

    for m in new_ones:
        if(m.Type == "helpful"):
            helpful_macpens_dict[m] = m.getLocId()
        if(m.Type == "tittat"):
            tittat_macpens_dict[m] = m.getLocId()
        if(m.Type == "unhelpful"):
            unhelpful_macpens_dict[m] = m.getLocId()
                
    
    #kill         
    if(iterations%24 == 0 and iterations != 0):
        print("--------------------killing-----------------------")
        list_del = []
        # for i, m in enumerate(macpen_list):
        #     if (m.food < ghost_val):
        #       list_del.append(m)
        #     else:
        #         m.food = m.food - ghost_val
        for m in helpful_macpens_dict:
            if(m.food < ghost_val):
                macpens_loc_dict[m.getLocId()].remove(m)
                list_del.append(m)
            else:
                m.food = m.food - ghost_val
        for m in tittat_macpens_dict:
            if(m.food < ghost_val):
                macpens_loc_dict[m.getLocId()].remove(m)
                list_del.append(m)
            else:
                m.food = m.food - ghost_val
        for m in unhelpful_macpens_dict:
            if(m.food < ghost_val):
                macpens_loc_dict[m.getLocId()].remove(m)
                list_del.append(m)
            else:
                m.food = m.food - ghost_val
                
        for i in list_del:
            try:
                del helpful_macpens_dict[i]
            except:
                pass
            try:
                del tittat_macpens_dict[i]
            except:
                pass
            try:
                del unhelpful_macpens_dict[i]
            except:
                pass
    
    screen.fill((0,0,0))
    for _, m in macpens_loc_dict.items():
        if(m != []):
            for m in m:
                m.draw(screen)
    for _, c in canteens_loc_dict.items():
        if(c != []):
            for c in c:
                c.draw(screen)
    
    screen.blit(time_surface, (0, 0))
    screen.blit(total_pop_surface, (0, 30))
    screen.blit(helpful_pop_surface, (width*block_size-149, 0))
    screen.blit(tittat_pop_surface, (width*block_size-171, 30))
    screen.blit(unhelpful_pop_surface, (width*block_size-170, 60))
    pygame.display.flip()
    pygame.time.delay(200)

pygame.quit()

================== Day:  0 Hour:  0 ================
Population:  300
Helpful Population:  100
Tittat Population:  100
Unhelpful Population:  100
================== Day:  0 Hour:  1 ================
Population:  300
Helpful Population:  100
Tittat Population:  100
Unhelpful Population:  100
================== Day:  0 Hour:  2 ================
Population:  300
Helpful Population:  100
Tittat Population:  100
Unhelpful Population:  100
================== Day:  0 Hour:  3 ================
Population:  300
Helpful Population:  100
Tittat Population:  100
Unhelpful Population:  100
================== Day:  0 Hour:  4 ================
Population:  300
Helpful Population:  100
Tittat Population:  100
Unhelpful Population:  100
================== Day:  0 Hour:  5 ================
Population:  300
Helpful Population:  100
Tittat Population:  100
Unhelpful Population:  100
================== Day:  0 Hour:  6 ================
Population:  300
Helpful Population:  100
Tittat Population:  100
Unhel

: 

: 

In [ ]:
# for m in helpful_macpens_dict:
#     print(m.ID)
#     print("length of history", len(m.history_list))

# for m in tittat_macpens_dict:
#     print(m.ID)
#     print("length of history", len(m.history_list))

# for m in unhelpful_macpens_dict:
#     print(m.ID)
#     print("length of history", len(m.history_list))